In [16]:
import networkx as nx
import numpy as np
import pandas as pd

In [17]:
df= pd.read_csv(r"D:\Documenti\DSE\Network science Prof. Gaito\Ntw. Proj\P_Data_Extract_From_Global_Bilateral_Migration.csv")

In [18]:
df1=df[df["Country Origin Name"] != 'Sierra Leone'].dropna().drop(["Migration by Gender Name", "Migration by Gender Code"], axis=1)
df1 #drop nulls and drop country Sierra leone due to inconsistencies

,Country Origin Name,Country Origin Code,Country Dest Name,Country Dest Code,1960 [1960],1980 [1980],2000 [2000]
0,Andorra,AND,Andorra,AND,0,0,0
1,Andorra,AND,Afghanistan,AFG,0,0,0
2,Andorra,AND,United Arab Emirates,ARE,0,0,0
3,Andorra,AND,Albania,ALB,0,0,0
4,Andorra,AND,Argentina,ARG,0,7,0
...,...,...,...,...,...,...,...
12427,Zimbabwe,ZWE,"Venezuela, RB",VEN,0,4,13
12428,Zimbabwe,ZWE,Vietnam,VNM,0,0,2
12429,Zimbabwe,ZWE,South Africa,ZAF,26957,74455,129558
12430,Zimbabwe,ZWE,Zambia,ZMB,90416,49691,31538


In [19]:
index=df1[(df1.iloc[:,0] == "Egypt, Arab Rep.") | (df1.iloc[:,0]=="Iran, Islamic Rep.") | (df1.iloc[:,0]=="Macedonia, FYR") |
         (df1.iloc[:,0] == "Taiwan, China") | (df1.iloc[:,0]=="Venezuela, RB") | (df1.iloc[:,2] == "Egypt, Arab Rep.") |
         (df1.iloc[:,2]=="Iran, Islamic Rep.") | (df1.iloc[:,2]=="Macedonia, FYR") | (df1.iloc[:,2] == "Taiwan, China") |
         (df1.iloc[:,2]=="Venezuela, RB")].index #countries to be dropped since nodes attributes were not available
data=df1.drop(index) 
data

,Country Origin Name,Country Origin Code,Country Dest Name,Country Dest Code,1960 [1960],1980 [1980],2000 [2000]
0,Andorra,AND,Andorra,AND,0,0,0
1,Andorra,AND,Afghanistan,AFG,0,0,0
2,Andorra,AND,United Arab Emirates,ARE,0,0,0
3,Andorra,AND,Albania,ALB,0,0,0
4,Andorra,AND,Argentina,ARG,0,7,0
...,...,...,...,...,...,...,...
12426,Zimbabwe,ZWE,Uzbekistan,UZB,0,0,0
12428,Zimbabwe,ZWE,Vietnam,VNM,0,0,2
12429,Zimbabwe,ZWE,South Africa,ZAF,26957,74455,129558
12430,Zimbabwe,ZWE,Zambia,ZMB,90416,49691,31538


In [20]:
x=list(data["Country Origin Name"].unique()) #add nodes to an empty directed graph
G=nx.DiGraph()
G.add_nodes_from(x)
G.order()

106

In [25]:
data1 = pd.read_csv(r"D:\Documenti\DSE\Network science Prof. Gaito\Ntw. Proj\country_long_lat.csv")
data2=data1.drop(data1.columns[[0, 4, 5, 6, 7]], axis=1) #drop unnecessary columns from the location csv file 
data3 = data2[data2['country'].isin(x)] #keep only the countries matching the first dataset list of countries 
data3

,latitude,longitude,country
0,42.546245,1.601554,Andorra
1,23.424076,53.847818,United Arab Emirates
2,33.939110,67.709953,Afghanistan
5,41.153332,20.168331,Albania
10,-38.416097,-63.616672,Argentina
...,...,...,...
229,41.377491,64.585262,Uzbekistan
235,14.058324,108.277199,Vietnam
242,-30.559482,22.937506,South Africa
243,-13.133897,27.849332,Zambia


In [26]:
y=data3.set_index('country').T.to_dict('dict') #create a dictionary out of the df for setting the nodes attributes
nx.set_node_attributes(G, y)

In [27]:
data00=data.drop(["Country Origin Code", "Country Dest Code","1960 [1960]", "1980 [1980]"], axis=1) #drop unn. cols
data00_final=data00[data00["2000 [2000]"] != '..'] #drop other missing values
data00_final["2000 [2000]"]= data00_final["2000 [2000]"].astype(str).astype(int) #transform migration flows into integers
data00_final1=data00_final[data00_final["2000 [2000]"] != 0] #so as to drop null values
data00_final1

C:\Users\Ivo\AppData\Local\Temp\ipykernel_2512\1010579244.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data00_final["2000 [2000]"]= data00_final["2000 [2000]"].astype(str).astype(int) #transform migration flows into integers


,Country Origin Name,Country Dest Name,2000 [2000]
5,Andorra,Australia,6
8,Andorra,Belgium,3
10,Andorra,Bolivia,2
13,Andorra,Canada,15
15,Andorra,Switzerland,16
...,...,...,...
12423,Zimbabwe,Uganda,7
12424,Zimbabwe,United States,12203
12428,Zimbabwe,Vietnam,2
12429,Zimbabwe,South Africa,129558


In [28]:
print("the total number of migrants is", data00_final1["2000 [2000]"].sum())

the total number of migrants is 113511192


In [29]:
values=data00_final1.to_records(index=False).tolist() #make a list of lists inlcuding source, target and edge attribute
values
G.add_weighted_edges_from(values)
nx.write_gexf(G, path="D:\\Documenti\\DSE\\Network science Prof. Gaito\\Ntw. Proj\\G60.gexf")


In [30]:
print(G.order(), G.size())

106 7889
